# YOLO Fine-tuning for Furniture Detection

This notebook demonstrates how to fine-tune a YOLO model to detect specific furniture classes defined in the project configuration.

## Steps:
1.  **Setup**: Install necessary libraries (`fiftyone`, `ultralytics`).
2.  **Data Preparation**: 
    *   Define the target classes (from `src/config.py`).
    *   Download a subset of the COCO-2017 dataset containing these classes using `fiftyone`.
    *   Filter the dataset to keep only the relevant labels.
    *   Export the dataset in YOLO format.
3.  **Training**:
    *   Load a pre-trained YOLO model (e.g., `yolo11n.pt`).
    *   Fine-tune the model on the custom furniture dataset.


In [ ]:
%pip install fiftyone ultralytics

In [2]:
import fiftyone as fo
import fiftyone.zoo as foz
from ultralytics import YOLO
import os

# Define the classes we want to detect (from src/config.py)
classes = [
    'chair', 'couch', 'bed', 'dining table', 'toilet',
    'tv', 'laptop', 'mouse', 'keyboard', 'microwave',
    'oven', 'toaster', 'sink', 'refrigerator', 'book',
    'clock', 'vase', 'potted plant'
]

print(f"Target classes: {classes}")

Target classes: ['chair', 'couch', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'keyboard', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'potted plant']


In [3]:
# Download and filter COCO-2017 dataset
# We use a subset (max_samples) for demonstration. 
# Remove max_samples to download all matching images for a "large-ish" dataset.

print("Downloading/Loading training data...")
dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    label_types=["detections"],
    classes=classes,
    max_samples=2000, # Limit for demo purposes
    # dataset_name="coco-furniture-train" # Optional: name the dataset in fiftyone
)

print("Downloading/Loading validation data...")
val_dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections"],
    classes=classes,
    max_samples=500, # Limit for demo purposes
    # dataset_name="coco-furniture-val"
)

# Filter the labels to ONLY include the classes we want
# (load_zoo_dataset downloads images with at least one instance, but might include other labels)
# We use filter_labels to ensure only our target classes are kept in the ground_truth
from fiftyone import ViewField as F

dataset = dataset.filter_labels("ground_truth", F("label").is_in(classes))
val_dataset = val_dataset.filter_labels("ground_truth", F("label").is_in(classes))

print(f"Train set size: {len(dataset)}")
print(f"Val set size: {len(val_dataset)}")

Downloading/Loading training data...
 100% |██████|    1.9Gb/1.9Gb [1.6m elapsed, 0s remaining, 13.1Mb/s]       
Extracting annotations to '/Users/m3/fiftyone/coco-2017/raw/instances_train2017.json'
 100% |██████|    1.9Gb/1.9Gb [1.6m elapsed, 0s remaining, 13.1Mb/s]       
Extracting annotations to '/Users/m3/fiftyone/coco-2017/raw/instances_train2017.json'
 100% |████████████████| 2000/2000 [3.5m elapsed, 0s remaining, 8.8 images/s]       
 100% |████████████████| 2000/2000 [3.5m elapsed, 0s remaining, 8.8 images/s]       
Writing annotations for 2000 downloaded samples to '/Users/m3/fiftyone/coco-2017/train/labels.json'
Writing annotations for 2000 downloaded samples to '/Users/m3/fiftyone/coco-2017/train/labels.json'
Dataset info written to '/Users/m3/fiftyone/coco-2017/info.json'
Dataset info written to '/Users/m3/fiftyone/coco-2017/info.json'
Loading 'coco-2017' split 'train'
Loading 'coco-2017' split 'train'
 100% |███████████████| 2000/2000 [6.0s elapsed, 0s remaining, 213.0 sa

In [4]:
# Export to YOLO format
export_dir = os.path.abspath("../data/coco_furniture")
label_field = "ground_truth"

print(f"Exporting dataset to {export_dir}...")

# Export training set
dataset.export(
    export_dir=export_dir,
    dataset_type=fo.types.YOLOv5Dataset,
    label_field=label_field,
    classes=classes,
    split="train"
)

# Export validation set
val_dataset.export(
    export_dir=export_dir,
    dataset_type=fo.types.YOLOv5Dataset,
    label_field=label_field,
    classes=classes,
    split="val"
)

print("Export complete.")

Exporting dataset to /Users/m3/local-docs/uni-related/floor_plan_from_image/data/coco_furniture...
 100% |███████████████| 2000/2000 [2.6s elapsed, 0s remaining, 847.2 samples/s]      
Directory '/Users/m3/local-docs/uni-related/floor_plan_from_image/data/coco_furniture' already exists; export will be merged with existing files
 100% |███████████████| 2000/2000 [2.6s elapsed, 0s remaining, 847.2 samples/s]      
Directory '/Users/m3/local-docs/uni-related/floor_plan_from_image/data/coco_furniture' already exists; export will be merged with existing files
 100% |█████████████████| 500/500 [625.7ms elapsed, 0s remaining, 799.1 samples/s]      
Export complete.
 100% |█████████████████| 500/500 [625.7ms elapsed, 0s remaining, 799.1 samples/s]      
Export complete.


In [5]:
# Verify the dataset.yaml
yaml_path = os.path.join(export_dir, "dataset.yaml")

# FiftyOne's YOLO export might not create the exact yaml structure Ultralytics expects for 'path'
# We might need to adjust it or just pass the path to the yaml file if it's correct.
# Let's inspect it.
with open(yaml_path, 'r') as f:
    print(f.read())
    
# Sometimes we need to ensure the 'path' in yaml is absolute or relative correctly.
# Ultralytics usually handles it if we pass the absolute path to the yaml.

names:
  0: chair
  1: couch
  2: bed
  3: dining table
  4: toilet
  5: tv
  6: laptop
  7: mouse
  8: keyboard
  9: microwave
  10: oven
  11: toaster
  12: sink
  13: refrigerator
  14: book
  15: clock
  16: vase
  17: potted plant
path: /Users/m3/local-docs/uni-related/floor_plan_from_image/data/coco_furniture
train: ./images/train/
val: ./images/val/



In [6]:
# Train the model
# Load the model (using the one in models/yolo/yolo11n.pt if available, else download)
model_path = "../models/yolo/yolo11n.pt"
if not os.path.exists(model_path):
    print(f"Model not found at {model_path}, using 'yolo11n.pt' (will download)")
    model_name = "yolo11n.pt"
else:
    print(f"Loading model from {model_path}")
    model_name = model_path

model = YOLO(model_name)

# Train
# We specify the data config we just exported
# epochs=10 is a starting point, increase for better results
results = model.train(
    data=yaml_path, 
    epochs=10, 
    imgsz=640,
    project="../experiments/yolo_training",
    name="furniture_finetune"
)

print("Training complete.")

Loading model from ../models/yolo/yolo11n.pt
New https://pypi.org/project/ultralytics/8.3.231 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.221 🚀 Python-3.11.9 torch-2.6.0 CPU (Apple M3)
New https://pypi.org/project/ultralytics/8.3.231 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.221 🚀 Python-3.11.9 torch-2.6.0 CPU (Apple M3)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/m3/local-docs/uni-related/floor_plan_from_image/data/coco_furniture/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, 

Action failed <400/12: projects.create/v1.0 (Validation error (Project name cannot be blank))> (name=../experiments/yolo_training, description=, method=get)
Action failed <400/12: projects.create/v1.0 (Validation error (Project name cannot be blank))> (name=../experiments/yolo_training, description=, method=get)
Action failed <400/12: projects.create/v1.0 (Validation error (Project name cannot be blank))> (name=../experiments/yolo_training, description=, method=get)


WARNING ⚠️ ClearML installed but not initialized correctly, not logging this run. Action failed <400/12: projects.create/v1.0 (Validation error (Project name cannot be blank))> (name=../experiments/yolo_training, description=, method=get)
Overriding model.yaml nc=80 with nc=18

                   from  n    params  module                                       arguments                     
Overriding model.yaml nc=80 with nc=18

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]              

2025/11/25 06:17:53 INFO mlflow.tracking.fluent: Experiment with name '../experiments/yolo_training' does not exist. Creating a new experiment.
2025/11/25 06:17:53 INFO mlflow.bedrock: Enabled auto-tracing for Bedrock. Note that MLflow can only trace boto3 service clients that are created after this call. If you have already created one, please recreate the client by calling `boto3.client`.
2025/11/25 06:17:53 INFO mlflow.tracking.fluent: Autologging successfully enabled for boto3.
2025/11/25 06:17:53 INFO mlflow.bedrock: Enabled auto-tracing for Bedrock. Note that MLflow can only trace boto3 service clients that are created after this call. If you have already created one, please recreate the client by calling `boto3.client`.
2025/11/25 06:17:53 INFO mlflow.tracking.fluent: Autologging successfully enabled for boto3.
2025/11/25 06:17:54 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/11/25 06:17:54 INFO mlflow.tracking.fluent: Autologging successfully e

MLflow: logging run_id(74f1806ee12d4f06aae7f4e5c49ebaee) to runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/m3/local-docs/uni-related/floor_plan_from_image/experiments/yolo_training/furniture_finetune
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/m3/local-docs/uni-related/floor_plan_from_image/experiments/yolo_training/furniture_finetune
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       1/10 

KeyboardInterrupt: 